# Label Vector for Films Nominated for The Best Animated Feature 

In [1]:
# import all the packages we will need

import imdb
from imdb import Cinemagoer
import requests
import pandas as pd
import bs4
from bs4 import BeautifulSoup

In [2]:
ia = Cinemagoer()

In [3]:
# for a given page url on imdb, this function will gather all the links for the searched films and return a list of 
# film ids of the form '/titls/tt########'

def gather_id_urls(url):
    id_urls = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    movies= []
    for a in soup.find_all('a', href=True):
        if ('/title/tt' in a['href']) & (a['href'] not in id_urls):
            id_urls.append(a['href'])    
    return id_urls

In [4]:
# given a list of imdb ids of the form '/title/tt######/' returns id of the form ######

def clean_ids_from_urls(id_urls):
    imdb_ids = []
    for ids in id_urls:
        imdb_ids.append(ids[9:-1])
    return imdb_ids

In [5]:
## creates a dataframe of academy award nominated animated feature films

def create_nominee_label(nominee_ids):
    title = []
    nomination = []
    for nominee in nominee_ids:
        title.append(ia.get_movie(nominee)['localized title'])
        nomination.append(ia.get_movie_awards(nominee)['data']['awards'][0]['notes'])
    binary_nomination = [1 if i[0:21] == 'Best Animated Feature' else 0 for i in nomination]
    d = {'imdb ids':nominee_ids, 'title': title, 'nomination':nomination, 'label':binary_nomination}
    return pd.DataFrame(d)

In [6]:
## imdb advanced search to find all animated feature films nominated for an academy award
## betwenn 2001 and 2021

url_nominees = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2001-01-01,2021-12-31&genres=animation&groups=oscar_nominee&runtime=40,&sort=year,asc&count=250&view=simple'

In [7]:
nominee_ids = clean_ids_from_urls(gather_id_urls(url_nominees))
labels_1 = create_nominee_label(nominee_ids)

In [18]:
labels_1['nomination'].value_counts()

Best Animated Feature Film of the Year                                    42
Best Animated Feature Film                                                22
Best Animated Feature                                                     14
Best Achievement in Music Written for Motion Pictures, Original Song       8
Best Music, Original Song                                                  3
Best Achievement in Music Written for Motion Pictures, Original Score      3
Best Achievement in Music Written for Motion Pictures (Original Song)      3
Best Achievement in Music Written for Motion Pictures (Original Score)     2
Best Achievement in Visual Effects                                         1
Name: nomination, dtype: int64

In [17]:
labels_1.iloc[14] 

imdb ids                         0317705
title                    The Incredibles
nomination    Best Animated Feature Film
label                                  1
Name: 14, dtype: object

In [15]:
labels_1.loc[labels_1['nomination'] == 'Best Music, Original Song']

,imdb ids,title,nomination,label
1,0198781,"Monsters, Inc.","Best Music, Original Song",0
8,0282120,The Wild Thornberrys,"Best Music, Original Song",0
10,0286244,The Triplets of Belleville,"Best Music, Original Song",0


In [12]:
best_animated_film_ids = labels_1['imdb ids'].loc[labels_1['label']==1].copy()

In [92]:
with open('animated_films_ids.csv', 'r') as f:
    all_animated_films = [line[0:-2] for line in f] #[0:-2] drops the ,\n

In [83]:
y = [(film_id in best_animated_film_ids.values)*1  for film_id in all_animated_films]

In [96]:
sum(y) == len(best_animated_film_ids)  #Sanity check but we will need to spot check the total dataframe.

True

In [25]:
imdb_ids

NameError: name 'imdb_ids' is not defined